# File Name: advanced_rag_building_part1.ipynb
### Location: Chapter 6
### Purpose: 
#####       1. Example of simple RAG pattern.
#####       2. Example of HyDe RAG pattern.
#####       3. Example of multi query RAG pattern.
#####       4. Example of LLM Augmented Retrieval RAG pattern.
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "PyPDF2" \
    "pypdf" \
    "llama-index" \
    "llama-index-llms-bedrock" \
    "llama-index-embeddings-bedrock" \
    "llama-index-embeddings-huggingface" \
    "llama-index-llms-langchain" \
    "langchain-chroma>=0.1.2" \
    "ipywidgets>=7.6.5" \
    "jupyterlab" \
    "jupyter" \
    "tqdm" \
    "iprogress>=0.4" \
    "ipynb" \
    "langchain-aws>=0.1.7"  

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import warnings
import time
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Import necessary modules
from langchain.prompts import PromptTemplate

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

# Find out data directory

#### 1. Retrieves the current working directory and prints it.
#### 2. Builds a path that navigates up one directory and appends 'data/rag_use_cases' to the path, then prints this resulting path.

In [ ]:
try:
    # Get the current working directory
    current_directory = os.getcwd()
    
    # Print the current working directory
    print(f"Current working directory: {current_directory}")
    
    # Attempt to navigate up one directory and then to 'data/rag_use_cases'
    data_directory = os.path.join(os.path.dirname(current_directory), 'data/rag_use_cases')
    
    # Print the resulting path
    print(f"Data directory path: {data_directory}")
    
except FileNotFoundError as e:
    # Handle the case where the directory path does not exist
    print(f"Error: The specified path does not exist - {e}")
    
except Exception as e:
    # General exception handler for any other errors
    print(f"An unexpected error occurred: {e}")

# Disclaimer
##### Make Sure that data_directory is pointing to the right path and data files are present. Otherwise, you need to change the above code

# Define prompt, Amazon Bedrock Foundation model, and Amazon Bedrock embed model

In [ ]:
# Define prompt
prompt = "What is Amazon doing and cashflow?"

# List of Bedrock models with names and model codes
bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# List of Bedrock embed models with names and model codes
bedrock_embed_model_id = "amazon.titan-embed-text-v1"

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    # This session is responsible for managing credentials and region configuration
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name
    
    # Initialize Bedrock and Bedrock Runtime clients using Boto3
    # These clients will allow interactions with Bedrock-related AWS services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                          # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,  # Bedrock Runtime client instance
        "boto3_session": boto3_session                               # Current Boto3 session object
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print the error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")

# Prepare dataset 

##### 1.Parameter Inputs: Accepts data_directory (path to the PDF folder) and documents (list to store PDF data).
##### 2.File Loading Loop: Iterates through all files in the directory, checking if each one has a .pdf extension.
##### 3.PDF Loading with Error Handling: For each PDF, it uses PyPDFLoader to load the content, appending it to documents. If an error occurs, it prints an error message for that specific file, allowing the process to continue.
##### 4.Document Check and Output: If any documents are loaded, it prints the content of the first page from the first document; otherwise, it notifies that no PDFs were found.
##### 5.Function Return: Returns the updated documents list with loaded PDF content.

In [ ]:
def load_pdf_documents(data_directory, documents):
    """
    Load PDF documents from the specified directory and append their content to the documents list.

    Parameters:
    - data_directory (str): The directory path containing the PDF files.
    - documents (list): A list to store the loaded PDF data.

    Returns:
    - list: The updated documents list with the loaded PDF content.
    """
    # Loop through all files in the specified directory
    for filename in os.listdir(data_directory):
        if filename.endswith('.pdf'):  # Check if the file is a PDF
            file_path = os.path.join(data_directory, filename)
            try:
                # Initialize the PDF loader for the current file
                loader = PyPDFLoader(file_path)
                
                # Load the PDF data
                data = loader.load()
                
                # Extend the documents list with the loaded data
                documents.extend(data)
            except Exception as e:
                print(f"Error loading {filename}: {e}")  # Handle exceptions during loading

    # Display the content of the first page of the first document if available
    if documents:
        print(documents[0].page_content)  # Printing page content of the first document
    else:
        print("No PDF files found in the folder.")

    return documents

# Usage
documents = []
documents = load_pdf_documents(data_directory, documents)

# Modular Python setup for initializing language and embeddings models with error handling using Bedrock client.

### 1. initialize_language_model: This function initializes a language model using the Bedrock client and model ID. If successful, it returns the model; otherwise, it returns None.

### 2. initialize_embeddings_model: Similar to the first function, this initializes the embeddings model using the Bedrock client and embeddings model ID, with error handling in case of failure.

### 3. setup_models: This is the main function that calls the previous two functions to initialize both the language model and embeddings model. It handles errors, ensuring that if either initialization fails, the process is halted, and appropriate messages are shown.

In [ ]:
# Importing necessary packages from langchain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Function to initialize the language model with the Bedrock client
def initialize_language_model(client, model_id):
    """
    Initializes the language model using the provided Bedrock client and model ID.
    
    Args:
        client: The Bedrock client for model invocation.
        model_id (str): The ID of the language model to be initialized.
    
    Returns:
        ChatBedrock: The initialized language model or None if an error occurs.
    """
    try:
        # Initialize the language model
        llm = ChatBedrock(model_id=model_id, client=client)
        print("Successfully initialized the language model.")
        return llm
    except Exception as e:
        # Handle any errors during initialization
        print(f"Error initializing the language model: {e}")
        return None

# Function to initialize the Bedrock Embeddings model
def initialize_embeddings_model(client, embed_model_id):
    """
    Initializes the embeddings model using the provided Bedrock client and embeddings model ID.
    
    Args:
        client: The Bedrock client for model invocation.
        embed_model_id (str): The ID of the embeddings model to be initialized.
    
    Returns:
        BedrockEmbeddings: The initialized embeddings model or None if an error occurs.
    """
    try:
        # Initialize the embeddings model
        embeddings_model = BedrockEmbeddings(client=client, model_id=embed_model_id)
        print("Successfully initialized the Bedrock Embeddings model.")
        return embeddings_model
    except Exception as e:
        # Handle any errors during initialization
        print(f"Error initializing the Bedrock Embeddings model: {e}")
        return None

# Main function to set up both models with error handling
def setup_models(bedrock_client, bedrock_model_id, embed_model_id):
    """
    Sets up the language model and embeddings model, handling errors during the setup.
    
    Args:
        bedrock_client: The Bedrock client used to interact with the models.
        bedrock_model_id (str): The ID of the language model.
        embed_model_id (str): The ID of the embeddings model.
    
    Returns:
        tuple: (llm, embeddings_model) initialized models or None if an error occurs.
    """
    try:
        # Initialize the language model
        llm = initialize_language_model(bedrock_client, bedrock_model_id)
        if not llm:
            # Return None if language model initialization failed
            print("Failed to initialize language model, exiting setup.")
            return None, None

        # Initialize the embeddings model
        embeddings_model = initialize_embeddings_model(bedrock_client, embed_model_id)
        if not embeddings_model:
            # Return None if embeddings model initialization failed
            print("Failed to initialize embeddings model, exiting setup.")
            return llm, None

        print("Both models initialized successfully.")
        return llm, embeddings_model
    except Exception as e:
        # Handle any unexpected errors during the setup process
        print(f"Unexpected error in setup process: {e}")
        return None, None

# Example usage: setting up both models
llm, embeddings_model = setup_models(boto3_bedrock_runtime_client, bedrock_model_id, bedrock_embed_model_id)

if llm and embeddings_model:
    # Indicate that both models are ready for use
    print("Language model and embeddings model are ready for use.")
else:
    # Indicate that there was an issue with initialization
    print("One or both models failed to initialize.")

# Advanced RAG Patterns 

# 1. Simple Langchain Rag pattern

< Details >

# Modular document processing with error handling: splitting documents, creating embeddings, and storing in Chroma vector store.

### 1. split_documents:
##### Splits the provided documents into smaller chunks based on the specified chunk size and overlap.
##### If splitting fails, it returns None and prints an error message.

### 2. create_vectorstore:
##### Creates a vector store using Chroma, which stores the embeddings of the split documents.
##### If an error occurs during the vector store creation, it returns None and prints an error message.

### 3. create_ingest_vector_store:
##### Manages the overall process by calling split_documents and create_vectorstore in sequence.
##### If any step fails (splitting or vector store creation), it prints an error message and halts further execution.
##### Returns the created vector store if both steps are successful.

In [ ]:
# Function to split the documents into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Splits the provided documents into smaller chunks.

    Args:
        documents (list): List of documents to be split.
        chunk_size (int): The maximum size of each chunk (default is 1000).
        chunk_overlap (int): The overlap between consecutive chunks (default is 200).

    Returns:
        list: List of document chunks or None if an error occurs.
    """
    try:
        # Initialize a recursive character text splitter
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        
        # Split documents into smaller chunks
        splits = text_splitter.split_documents(documents)
        
        # Print the number of chunks created
        print(f"Number of splits: {len(splits)}")
        return splits
    except Exception as e:
        # Handle errors that occur during document splitting
        print(f"Error while splitting documents: {e}")
        return None

# Function to create embeddings and store vectors using Chroma
def create_vectorstore(splits, embeddings_model):
    """
    Creates a vector store from the document splits using an embeddings model.

    Args:
        splits (list): List of document chunks to be indexed.
        embeddings_model: The model used for embedding the documents.

    Returns:
        Chroma: The created vector store or None if an error occurs.
    """
    try:
        # Create a vector store using the splits and embeddings model
        vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)
        
        # Print success message
        print("Vector store created successfully.")
        return vectorstore
    except Exception as e:
        # Handle errors during vector store creation
        print(f"Error while creating vector store: {e}")
        return None

# Main execution function to modularize the flow
def create_ingest_vector_store(documents, embeddings_model):
    """
    Main function to manage the ingestion and creation of vector store.

    Args:
        documents (list): List of documents to process.
        embeddings_model: The model used for embedding documents into vectors.

    Returns:
        Chroma: The vector store created from the documents or None if an error occurs.
    """
    try:
        # Step 1: Split documents into chunks
        splits = split_documents(documents)
        
        # Check if document splitting was successful
        if not splits:
            raise Exception("Document splitting failed")

        # Step 2: Create a vector store using the embeddings model
        vectorstore = create_vectorstore(splits, embeddings_model)
        
        # Check if vector store creation was successful
        if not vectorstore:
            raise Exception("Vector store creation failed")

        # If all steps succeed
        print("Process completed successfully.")
        return vectorstore
        
    except Exception as e:
        # Handle any errors during the overall process
        print(f"An error occurred: {e}")
        return None

# Example usage
vectorstore_chroma = create_ingest_vector_store(documents, embeddings_model)


# Modular functions for creating a prompt template, setting up a retrieval QA system, executing a query, and managing the entire process with error handling.

### 1. create_prompt_template(): Defines the assistant's prompt template for generating responses, with error handling for template creation.
### 2. setup_retrieval_qa(): Configures the RetrievalQA chain using the LLM, vector store, and prompt template, including error handling for setup failures.
### 3. execute_query(): Executes a query on the retrieval QA chain and returns the response, with error handling for execution failures.
### 4. main_retrieval_qa(): Coordinates the entire process: creating the prompt, setting up the retrieval QA system, and executing the query, with error handling at each step.


In [ ]:
# Define the prompt template for the assistant
def create_prompt_template():
    try:
        # Template instructing the assistant on response generation
        prompt_template = """
            Human: Please use the following context to provide a clear and concise answer to the question below. 
            If the answer is unknown, simply state that you don't know, without attempting to guess.
            
            <context>
            {context}
            </context>

            Question: {question}

            Assistant:
        """
        prompt = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )
        print("Prompt template created successfully.")
        return prompt
    except Exception as e:
        # Handle errors during prompt template creation
        print(f"Error creating prompt template: {e}")
        return None

# Configure the retrieval-based question-answering chain
def setup_retrieval_qa(llm, vectorstore_chroma, prompt):
    try:
        # Set up the RetrievalQA chain using the LLM, vector store, and prompt
        retrievalqa_res = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore_chroma.as_retriever(
                search_type="similarity", search_kwargs={"k": 3}
            ),
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt}
        )
        print("RetrievalQA chain set up successfully.")
        return retrievalqa_res
    except Exception as e:
        # Handle errors during the setup of the RetrievalQA chain
        print(f"Error setting up RetrievalQA: {e}")
        return None

# Execute the query on the retrieval QA chain
def execute_query(retrievalqa_res, prompt):
    try:
        # Run the query with the provided prompt
        response = retrievalqa_res({"query": prompt})
        print("Query executed successfully.")
        return response
    except Exception as e:
        # Handle errors during query execution
        print(f"Error executing query: {e}")
        return None

# Main process for setting up and running the RetrievalQA chain
def main_retrieval_qa(llm, vectorstore_chroma, prompt_text):
    try:
        # Step 1: Create the prompt template
        prompt = create_prompt_template()
        if not prompt:
            print("Failed to create prompt template. Exiting.")
            return

        # Step 2: Set up the RetrievalQA chain
        retrievalqa_res = setup_retrieval_qa(llm, vectorstore_chroma, prompt)
        if not retrievalqa_res:
            print("Failed to set up RetrievalQA chain. Exiting.")
            return

        # Step 3: Execute the query with the provided prompt text
        response = execute_query(retrievalqa_res, prompt_text)
        if response:
            print("Final Response:", response)
        else:
            print("No response returned.")

    except Exception as e:
        # Handle any unexpected errors during the main process
        print(f"An unexpected error occurred in the main process: {e}")

# Example usage
main_retrieval_qa(llm, vectorstore_chroma, prompt)


# 2. HYDE patterns

< Details >

#### Refer:
https://arxiv.org/pdf/2212.10496.pdf https://github.com/langchainai/langchain/blob/master/cookbook/hypothetical_document_embeddings.ipynb

In [ ]:
# Python important package 
from langchain.chains import HypotheticalDocumentEmbedder

# Modular function to define a prompt template and replace placeholders with context and question values, with error handling for key errors and other exceptions.

### 1. Prompt Template Definition:
#### The hyde_prompt_template is a string that contains placeholders for the context and question. It instructs the assistant to provide concise answers or state "I don't know" if the answer is not available.

### 2. Function to Format the Prompt:
#### The get_refined_prompt_template() function takes in the prompt template, context, and question as inputs.
#### It uses the format() method to replace the placeholders ({context} and {question}) in the template with the provided context and question.
#### The function returns the formatted prompt or None in case of an error.

In [ ]:
# Define the prompt template with placeholders for context and question
hyde_prompt_template = """
    Human: Use the following pieces of context to provide a concise answer to the question at the end. 
    If you don't know the answer, just say that you don't know; don't try to make up an answer.
    <context>
    {context}
    </context>

    Question: {question}

    Assistant:
"""

def get_refined_prompt_template(prompt_template, context, question):
    """
    Replaces placeholders in the provided prompt template with the given context and question.

    Parameters:
    - prompt_template (str): The template string containing placeholders for context and question.
    - context (str): The context to be inserted into the template.
    - question (str): The question to be inserted into the template.
    
    Returns:
    - str or None: The prompt with context and question inserted, or None if an error occurs.
    """
    try:
        # Substitute the placeholders with context and question values
        formatted_prompt = prompt_template.format(context=context, question=question)
        return formatted_prompt
    except KeyError as e:
        print(f"Key error formatting prompt template: Missing placeholder - {e}")
        return None
    except Exception as e:
        print(f"Error formatting prompt template: {e}")
        return None

# The code defines functions to format a payload, invoke the Bedrock model, and generate a response based on user and assistant prompts with error handling at each step.

### 1. format_bedrock_payload:
##### This function formats the JSON payload for invoking the Bedrock model. It includes parameters like:
##### user_prompt: The user's input.
##### assistant_prompt: The assistant's previous response or context.
##### max_tokens, temperature, and top_p: Model configuration settings.
##### It constructs the payload in the required format and returns it as a JSON string. Errors are caught and logged.

### 2. invoke_bedrock_model:
##### This function uses the boto3 client to invoke the Bedrock model with the formatted payload.
##### It makes an API call to the model and retrieves the response.
##### The function extracts the text content from the response and returns it. If an error occurs, it logs the error and returns None.

### 3. call_bedrock:
##### This function orchestrates the process by:
##### Calling format_bedrock_payload to create the payload.
##### Passing the payload to invoke_bedrock_model for model invocation.
##### Returning the generated response from the model.
##### It checks for successful payload creation and model invocation, handling errors gracefully.

In [ ]:
def format_bedrock_payload(user_prompt, assistant_prompt, max_tokens=512, temperature=0.5, top_p=1.0):
    """
    Formats the payload for Bedrock model invocation.

    Parameters:
    - user_prompt (str): The user's input.
    - assistant_prompt (str): The assistant's previous response or context.
    - max_tokens (int): Maximum number of tokens in the response.
    - temperature (float): Sampling temperature.
    - top_p (float): Nucleus sampling parameter.

    Returns:
    - str: JSON payload for Bedrock model invocation.
    """
    try:
        messages = [
            {"role": 'user', "content": [{'type': 'text', 'text': user_prompt}]},
            {"role": 'assistant', "content": [{'type': 'text', 'text': assistant_prompt}]}
        ]
        
        payload = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temperature,
            "top_p": top_p
        })
        
        return payload
    except Exception as e:
        print(f"Error formatting Bedrock payload: {e}")
        return None

def invoke_bedrock_model(boto3_client, payload, model_id, accept='application/json', content_type='application/json'):
    """
    Invokes the Bedrock model with the given payload.

    Parameters:
    - boto3_client (boto3 client): Bedrock runtime client.
    - payload (str): JSON payload for the model.
    - model_id (str): Model ID for Bedrock.
    - accept (str): Accept header.
    - content_type (str): Content-Type header.

    Returns:
    - str: Text response from the model, or None if an error occurs.
    """
    try:
        response = boto3_client.invoke_model(
            body=payload,
            modelId=model_id,
            accept=accept,
            contentType=content_type
        )
        
        response_body = json.loads(response.get('body').read())
        response_text = response_body.get('content')[0]['text']
        
        return response_text
    except Exception as e:
        print(f"Error invoking Bedrock model: {e}")
        return None

def call_bedrock(user_prompt, assistant_prompt, model_id=bedrock_model_id):
    """
    Generates a response from the Bedrock model based on the user and assistant prompts.

    Parameters:
    - user_prompt (str): The user's input prompt.
    - assistant_prompt (str): The assistant's response or initial prompt.
    - model_id (str): The Bedrock model ID.

    Returns:
    - str: The response generated by the Bedrock model, or None if an error occurs.
    """
    # Step 1: Format the payload
    payload = format_bedrock_payload(user_prompt, assistant_prompt)
    if payload is None:
        print("Failed to create payload.")
        return None

    # Step 2: Invoke the model with the formatted payload
    response_text = invoke_bedrock_model(boto3_bedrock_runtime_client, payload, model_id)
    
    # Step 3: Check and return the model's response
    if response_text:
        print("Response received successfully.")
    else:
        print("No response received from the model.")
    
    return response_text

# A function to embed a user query, search for relevant documents, create a refined prompt, and retrieve a response from the Bedrock model, with error handling for key and general exceptions.

### 1. Initialize Embeddings Model:
##### Uses HypotheticalDocumentEmbedder to create an embedding model (hyde_embeddings) from the provided LLM and embeddings model, specifically for web search context.

### 2. Embed User Query:
##### Embeds the user’s input (prompt) into a vector representation using embed_query.

### 3. Document Similarity Search:
##### Performs a similarity search in the vectorstore_chroma to find the top 5 relevant documents based on the embedded query vector.

### 4. Context Creation:
##### Extracts the content from the relevant documents and constructs a context string to provide more information for the model's response.

### 5. Generate Refined Prompt:
##### Calls get_refined_prompt_template to create a refined prompt incorporating the context and the original user prompt.

### 6. Call Bedrock Model:
##### Uses the call_bedrock function to send the refined prompt to the Bedrock model and retrieve a response.

In [ ]:
# Function to embed a query using HypotheticalDocumentEmbedder and search for relevant documents
def generate_response_from_bedrock(prompt, embeddings_model, vectorstore_chroma, llm, bedrock_model_id):
    try:
        # Step 1: Initialize the HypotheticalDocumentEmbedder
        hyde_embeddings = HypotheticalDocumentEmbedder.from_llm(llm, embeddings_model, "web_search")
        print("Embeddings model initialized successfully.")
        
        # Step 2: Embed the user query
        result = hyde_embeddings.embed_query(prompt)
        print("Query embedded successfully.")
        
        # Step 3: Perform similarity search using the vectorstore
        relevant_docs = vectorstore_chroma.similarity_search_by_vector(result, k=5)
        print(f"Found {len(relevant_docs)} relevant documents.")
        
        # Step 4: Create the context from the relevant documents
        context = ""
        for doc in relevant_docs:
            context = f"{context} \n {doc.page_content}"
        
        # Step 5: Get the refined prompt with the context
        prompt_template = get_refined_prompt_template(hyde_prompt_template, context, prompt)
        if not prompt_template:
            print("Failed to create prompt template. Exiting.")
            return None
        print("Refined prompt template created successfully.")
        
        # Step 6: Call the Bedrock model to get the response
        response = call_bedrock(prompt_template, "Assistant:", bedrock_model_id)
        
        if response:
            print("Response received successfully.")
        else:
            print("No response received from Bedrock model.")
        
        return ( response, prompt_template )

    except KeyError as e:
        # Handle errors related to missing keys
        print(f"Key error: {e}")
        return None
    except Exception as e:
        # Handle any other unexpected errors
        print(f"An error occurred: {e}")
        return None

# Example usage
( response, prompt_template ) = generate_response_from_bedrock(prompt, embeddings_model, vectorstore_chroma, llm, bedrock_model_id)
print(response)

# End-to-end process for generating responses from Bedrock using HyDE embeddings, document retrieval, and prompt generation.

### 1. Initialize HyDE Embeddings using the provided LLM and embeddings model.
### 2. Generate an Embedding for the input prompt using HyDE.
### 3. Search for Relevant Documents in the vector store based on the embedding.
### 4. Compile Context from the retrieved documents.
### 5. Generate a Prompt with the compiled context and question.
### 6. Call Bedrock with the generated prompt and obtain a response.
### 7. The process includes error handling for each step to ensure robustness.

In [ ]:
# Assume necessary imports for HypotheticalDocumentEmbedder, vectorstore_chroma, etc., are already handled

def initialize_hyde_embeddings(llm, embeddings_model, task="web_search"):
    """
    Initializes the Hypothetical Document Embedder (HyDE) with the given LLM and embeddings model.
    
    Parameters:
    - llm: Language model to use for HyDE.
    - embeddings_model: Embeddings model for HyDE.
    - task (str): Task type for the embedder, default is 'web_search'.
    
    Returns:
    - HypotheticalDocumentEmbedder instance or None if an error occurs.
    """
    try:
        hyde_embeddings = HypotheticalDocumentEmbedder.from_llm(llm, embeddings_model, task)
        print("HyDE embeddings model initialized successfully.")
        return hyde_embeddings
    except Exception as e:
        print(f"Error initializing HyDE embeddings: {e}")
        return None

def generate_embedding(hyde_embeddings, prompt):
    """
    Generates an embedding for a given prompt using the initialized HyDE embeddings.
    
    Parameters:
    - hyde_embeddings: Initialized HyDE embeddings model.
    - prompt (str): The prompt for which to generate embeddings.
    
    Returns:
    - Embedding vector or None if an error occurs.
    """
    try:
        result = hyde_embeddings.embed_query(prompt)
        print("Embedding generated successfully.")
        return result
    except Exception as e:
        print(f"Error generating embedding for prompt: {e}")
        return None

def search_relevant_documents(vectorstore, embedding_vector, top_k=5):
    """
    Searches for relevant documents in the vector store based on the embedding vector.
    
    Parameters:
    - vectorstore: The vector store to search in.
    - embedding_vector: The embedding vector used for similarity search.
    - top_k (int): Number of top documents to retrieve.
    
    Returns:
    - List of retrieved documents or an empty list if an error occurs.
    """
    try:
        relevant_docs = vectorstore.similarity_search_by_vector(embedding_vector, k=top_k)
        print(f"Retrieved {len(relevant_docs)} relevant documents.")
        return relevant_docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

def compile_context_from_docs(relevant_docs):
    """
    Compiles a context string from the retrieved documents.
    
    Parameters:
    - relevant_docs (list): List of documents with page content.
    
    Returns:
    - Compiled context string.
    """
    context = ""
    for doc in relevant_docs:
        context += f"\n {doc.page_content}"
    print("Context compiled from documents.")
    return context

def generate_prompt(prompt_template, context, question):
    """
    Generates a complete prompt by formatting the template with the context and question.
    
    Parameters:
    - prompt_template (str): The template string.
    - context (str): The compiled context.
    - question (str): The question for the assistant.
    
    Returns:
    - Formatted prompt or None if an error occurs.
    """
    try:
        return get_refined_prompt_template(prompt_template, context, question)
    except Exception as e:
        print(f"Error generating prompt: {e}")
        return None

# Main function to execute the process
def main_process(prompt, llm, embeddings_model, vectorstore, prompt_template, bedrock_model_id):
    """
    Executes the end-to-end process of initializing HyDE, generating embeddings, 
    retrieving documents, compiling context, generating prompt, and calling Bedrock.
    
    Parameters:
    - prompt (str): User's initial query.
    - llm: Language model for HyDE.
    - embeddings_model: Embeddings model for HyDE.
    - vectorstore: Vector store for document retrieval.
    - prompt_template (str): Template for prompt generation.
    - bedrock_model_id (str): Model ID for Bedrock.
    """
    try:
        # Step 1: Initialize HyDE embeddings
        hyde_embeddings = initialize_hyde_embeddings(llm, embeddings_model)
        if not hyde_embeddings:
            return

        # Step 2: Generate embedding for the prompt
        embedding_vector = generate_embedding(hyde_embeddings, prompt)
        if not embedding_vector:
            return

        # Step 3: Retrieve relevant documents
        relevant_docs = search_relevant_documents(vectorstore, embedding_vector)
        if not relevant_docs:
            print("No relevant documents found.")
            return

        # Step 4: Compile context from retrieved documents
        context = compile_context_from_docs(relevant_docs)

        # Step 5: Generate final prompt
        full_prompt = get_refined_prompt_template(prompt_template, context, prompt)
        if not full_prompt:
            return

        # Step 6: Call Bedrock and get response
        response = call_bedrock(full_prompt, "Assistant:", bedrock_model_id)

        # Step 7: Print response
        if response:
            print("Final Response:\n", response)
        else:
            print("No response received from Bedrock.")
    except Exception as e:
        print(f"An unexpected error occurred in the main process: {e}")

# Run main process with given parameters
main_process(
    prompt,
    llm,  # Assuming llm instance is provided
    embeddings_model,  # Assuming embeddings model instance is provided
    vectorstore_chroma,  # Assuming vector store instance is provided
    prompt_template,
    bedrock_model_id
)

# 3. Multi Query Retrieval

< Details >

#### Refer:
https://arxiv.org/abs/2402.03367

### The multi_query_prompt_template is designed to generate five distinct variations of a user's question to improve document retrieval accuracy in vector databases. By framing the question from multiple perspectives, this approach aims to overcome the limitations of similarity-based matching. The mq_prompt_template then guides the assistant to use the provided context to answer a question directly, ensuring concise, accurate responses or acknowledging when an answer is unknown.

In [ ]:
multi_query_prompt_template = """
    You are an AI language model assistant. Your task is to create five unique variations of the user’s question to 
    help retrieve relevant documents from a vector database. By presenting multiple perspectives on the user’s question, 
    you aim to enhance the search process and address some limitations of distance-based similarity matching. 
    Please provide each alternative question separated by newlines only.

    For example, these alternative questions:

    'What is Amazon's current cash flow status?'
    'Can you provide an overview of Amazon's cash flow performance?'
    'How has Amazon's cash flow trended in recent quarters?'
    'What factors are impacting Amazon's cash flow?'
    'What is Amazon’s approach to managing cash flow?'

    Not:

    'What are Amazon's main business segments and services?'
    'How has Amazon's approach to innovation impacted its market position?'
    'What are some recent developments in Amazon’s technology and infrastructure?'
    'How does Amazon prioritize sustainability within its operations?'
    'What are Amazon's key strategies for customer satisfaction and loyalty?'

    Original question: {question}"""

mq_prompt_template = """

        Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
        <context>
        {context}
        </context

        Question: {question}
    """

# Functions to generate related queries and retrieve embeddings from AWS Bedrock, with error handling included.

### 1. multiple_related_queries_generator(question): Generates multiple related queries based on an input question by formatting a prompt, invoking a model on AWS Bedrock, and extracting relevant queries from the response.

### 2. get_embeddings(text): Retrieves an embedding vector for a given text input by preparing a payload, invoking the embedding model on AWS Bedrock, and extracting the embedding from the response.

In [ ]:
def multiple_related_queries_generator(question):
    """
    Generates multiple related queries based on the provided question by leveraging an AI model.

    Parameters:
    - question (str): The original user question for which related queries are generated.

    Returns:
    - list: A list of generated queries relevant to the input question.
    """
    try:
        queries_generated = []
        
        # Generate prompt template based on the original question
        template = get_refined_prompt_template(multi_query_prompt_template, "", question)
        
        # Call the model to generate alternative queries
        response_text = call_bedrock(
            user_prompt=template,  # User's query template
            assistant_prompt="Below are the generated Questions separated by \n:",  # Assistant's response prompt
            model_id=bedrock_model_id
        )

        # Split the response text into separate queries, ignoring blank lines
        queries_generated = [line for line in response_text.split("\n") if line.strip()]

        # Filter out short or irrelevant lines
        queries = [q for q in queries_generated if len(q) > 5]

        return queries
    except Exception as e:
        print(f"Error generating related queries: {e}")
        return []

def get_embeddings(text):
    """
    Retrieves the embedding for a given text using Bedrock's embedding model.

    Parameters:
    - text (str): The input text to generate embeddings for.

    Returns:
    - list or None: The embedding vector for the input text, or None if an error occurs.
    """
    try:
        # Prepare the input payload for the model
        body = json.dumps({"inputText": text})
        accept = 'application/json' 
        content_type = 'application/json'

        # Invoke the embedding model
        response = boto3_bedrock_runtime_client.invoke_model(
            body=body,
            modelId=bedrock_embed_model_id,
            accept=accept,
            contentType=content_type
        )
        
        # Process the response and retrieve the embedding
        response_body = json.loads(response['body'].read())
        embedding = response_body.get('embedding')
        
        if embedding:
            return embedding
        else:
            print("No embedding found in the response.")
            return None

    except Exception as e:
        print(f"Error generating embedding for text: {e}")
        return None


# Generating Related Queries and Retrieving Relevant Documents for Bedrock Model Responses

### This function generates related queries based on an initial prompt, retrieves relevant documents for each query using embeddings, compiles the document content, and formats it into a refined prompt for the Bedrock model to produce a response. The process involves calling an embedding function for each query, conducting a similarity search in a vector store, and appending any found documents to the relevant_documents string. Finally, it passes the compiled information to the Bedrock model, ensuring error handling at each step to manage any issues with query generation, document retrieval, or model invocation.

### 1. Generate Related Queries:
##### It begins by calling multiple_related_queries_generator(prompt), which generates several related queries based on the initial prompt. These queries will be used to search for relevant documents. An empty string relevant_documents is initialized to store document content gathered for each query.

### 2. Retrieve Relevant Documents for Each Query:
##### For each generated query, the code tries to:
##### Retrieve an embedding for the query using get_embeddings(query).
##### Use this embedding to search for relevant documents in the vectorstore_chroma vector store via similarity_search_by_vector(), limiting to the top result (k=1).
##### If documents are found, the code appends the content of each document to the relevant_documents string, organizing them as "Document 1", "Document 2," etc. If no results are found for a query, a message is printed.
##### Each document retrieval process includes its own try-except block to handle and report any errors.

### 3. Compile Documents and Generate Final Prompt:
##### After collecting document content from all queries, the code uses get_refined_prompt_template() to generate a new prompt template (question_answer_template). This template is based on the original prompt, the gathered relevant_documents content, and the specified mq_prompt_template.

### 4. Call Bedrock Model for Response:
##### The refined prompt template is passed to the Bedrock model via call_bedrock() to obtain the final response.
##### This function invocation specifies both the user prompt (question_answer_template) and an assistant response prefix (“Assistant:”), along with the model ID (bedrock_model_id).

In [ ]:
# Function to generate multiple related queries
try:
    # Generate multiple related queries based on the prompt
    queries = multiple_related_queries_generator(prompt)
    relevant_documents = ""  # Initialize empty string to store retrieved documents

    # Iterate over each generated query and retrieve relevant documents
    for i, query in enumerate(queries):
        try:
            # Retrieve embedding for each query and perform similarity search
            embedding = get_embeddings(query)
            search_results = vectorstore_chroma.similarity_search_by_vector(embedding, k=1)
            
            # Check if search results are not empty and add to relevant documents
            if search_results:
                relevant_documents += f"\n Document {i + 1}:\n" + str(search_results[0].page_content)
            else:
                print(f"No relevant documents found for query {i + 1}")

        except Exception as e:
            print(f"Error retrieving document for query {i + 1}: {e}")

    # Generate refined prompt template with the gathered documents and original prompt
    question_answer_template = get_refined_prompt_template(
        mq_prompt_template, relevant_documents, prompt
    )

    # Call Bedrock model with the generated prompt and retrieve response
    response = call_bedrock(
        question_answer_template, # User prompt
        "Assistant:", # Assistant's response prompt
        bedrock_model_id
    )

    # Print the response
    print(response)

except Exception as e:
    print(f"An error occurred in the query generation and retrieval process: {e}")

# 4. LLM Augmented Retrieval patterns

< Details >

#### Refer:
https://arxiv.org/pdf/2404.05825.pdf

# Function to Create and Manage LLM Prompt Templates with Error Handling

### Function to create a prompt template with error handling and demonstrates how to use it for generating a prompt with context and question variables. Below is a detailed breakdown of the flow:

### 1. Function Definition: create_llm_aug_prompt_template: The function create_llm_aug_prompt_template is designed to create a PromptTemplate using a template string and input variables.
### 2. Creating the PromptTemplate: The function tries to create a PromptTemplate by passing the template string and input_vars list to the PromptTemplate constructor.

In [ ]:
# Define a function to create the prompt template with error handling
def create_llm_aug_prompt_template(template, input_vars):
    """
    Creates a prompt template by substituting variables for context and question.
    
    Parameters:
    - template (str): The template string containing placeholders for context and question.
    - input_vars (list): A list of input variable names required by the template.
    
    Returns:
    - PromptTemplate or None: Returns the PromptTemplate if successful, None if an error occurs.
    """
    try:
        # Attempt to create the prompt template with the provided input variables
        prompt_template = PromptTemplate(template=template, input_variables=input_vars)
        print("Prompt template created successfully.")
        return prompt_template

    except Exception as e:
        # Handle errors during prompt template creation
        print(f"Error creating prompt template: {e}")
        return None


# Define the template string and input variables
llm_aug_prompt_template = """
    Human: Use the following pieces of context to provide a concise answer to the question at the end. 
    If you don't know the answer, just say that you don't know and don't try to make up an answer.
    <context>
    {context}
    </context>

    Question: {question}

    Assistant:
"""

# Create the prompt template using the function
PROMPT = create_llm_aug_prompt_template(llm_aug_prompt_template, ["context", "question"])

# Verify and use the created prompt template if successful
if PROMPT:
    print("Prompt template is ready for use.")
else:
    print("Prompt template creation failed.")

### set up a RetrievalQA system that combines a language model (LLM) with a retriever to perform question answering using relevant documents. Here's a breakdown of the steps and components involved:

### 1. RetrievalQA Instance Creation: The RetrievalQA.from_chain_type() method is used to create a RetrievalQA instance. This method combines a language model (LLM) with a retriever (in this case, using a Chroma Vector Store).

In [ ]:
try:
    # Create a RetrievalQA instance from the specified chain type and retriever
    retrievalqa = RetrievalQA.from_chain_type(
        llm=llm,  # Language model to be used
        chain_type="stuff",  # Chain type indicating how to process input/output
        retriever=vectorstore_chroma.as_retriever(  # Set up the retriever to use similarity search
            search_type="similarity",  # Define the search type for retrieving relevant documents
            search_kwargs={"k": 3}  # Set the number of top similar documents to retrieve
        ),
        return_source_documents=True,  # Ensure source documents are returned along with the response
        chain_type_kwargs={"prompt": PROMPT}  # Pass the prompt template for question-answer generation
    )

    # Execute the retrieval QA process with the provided query (prompt)
    response = retrievalqa({"query": prompt})

    # Print the response from the retrieval QA system
    print(response)

except Exception as e:
    # Handle any errors that may occur during the RetrievalQA process
    print(f"Error during RetrievalQA process: {e}")


# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html